In [2]:
import numpy as np
import os
from astroquery.jplhorizons import Horizons

In [3]:
# Paths to directories containing data
home_path = os.path.expanduser('~/')
path_67p = home_path+'Documents/year1/shape_modelling/67p/'

In [94]:
# Constants
km2au = 1.496e8
c = 3e5

In [82]:
### Function to create a file in the 'Mikko' format required for shape modelling 

def mikkoWrite(filename,day_list,int_list,sunx,suny,sunz,earthx,earthy,earthz, min_dist):
# Code to write to lightcurve file in Mikko format
# Additional argument to limit files to points beyond min_dist from the sun

    counter = 0
    fout = open(filename, 'w+')
    sameday = []
    for i, day in enumerate(day_list[:-1]):
        if (np.sqrt(sunx[i]**2+suny[i]**2+sunz[i]**2) >= min_dist):
            newday = True
            sameday.append(day)
            if abs(day-day_list[i+1])<0.5:
                newday = False
            if newday==False:
                if day not in sameday:
                    sameday.append(day)
            else:
                fout.write('%i 1\n'%(len(sameday)))
                for time in sameday:
                    loc = day_list.index(time)
                    fout.write('%.6f %.6e %.10f %.8f %.8f %.8f %.8f %.8f\n'%(time+2400000.5,int_list[loc],sunx[loc], suny[loc], sunz[loc], earthx[loc], earthy[loc], earthz[loc]))
                counter = counter+1
                sameday = []
    fout.write('1 1\n')
    fout.write('%.6f %.6e %.10f %.8f %.8f %.8f %.8f %.8f\n'%(day_list[-1]+2400000.5,int_list[-1],sunx[-1], suny[-1], sunz[-1], earthx[-1], earthy[-1], earthz[-1]))
    counter = counter+1
    fout.close()
    return counter

In [14]:
# Read LSST sim output

lsst_file_name = path_67p+'LSST_out_LC_67P.txt'
lsst_file = open(lsst_file_name, 'r')

lsst_mjd =[]; lsst_ra = []; lsst_dec = []
lsst_sunx = []; lsst_suny = []; lsst_sunz = []
lsst_phase_ang = [];
lines_lsst = lsst_file.readlines()
for line in lines_lsst[25:]:
    dat = line.split()
    lsst_mjd.append(float(dat[2]))
    lsst_ra.append(float(dat[5]))
    lsst_dec.append(float(dat[7]))
    lsst_sunx.append(float(dat[9]))
    lsst_suny.append(float(dat[10]))
    lsst_sunz.append(float(dat[11]))
    lsst_phase_ang.append(float(dat[21]))

lsst_sunx = np.array(lsst_sunx)/km2au
lsst_suny = np.array(lsst_suny)/km2au
lsst_sunz = np.array(lsst_sunz)/km2au

In [31]:
# File taken from columns in OG spreadsheet WITH < 3AU OBS REMOVED**
# (just did this in an excel spreadsheet)
#lc_file = path_67p+'LSST_3AU_sun_comet.dat'

#mjd = []
#sun_x = []; sun_y = []; sun_z = []

#f = open(lc_file, 'r')
#lines = f.readlines()
#for line in lines:
#    data=line.split()
#    mjd.append(float(data[0]))
#    sun_x.append(float(data[2]))
#    sun_y.append(float(data[3]))
#    sun_z.append(float(data[4]))
mjd[-1]+2400000.5

2463475.490745

In [90]:
mjd = np.array(lsst_mjd)
sun_x = lsst_sunx
sun_y = lsst_suny
sun_z = lsst_sunz

- - - - - - - - - - - - -

In [54]:
# LSST output only has obj-Sun geom: also need Earth-obj
# Also want list of solar phase angles, to compute abs mag from phase function

# Splitting this into 2 because Horizons wouldn't let me do all 655 epochs at once :(
obj = Horizons(id='90000693', location='500',epochs=mjd[:500])

In [55]:
earth_x = []; earth_y=[]; earth_z=[];
vec = obj.vectors(refplane='earth')  # LSST output in equatorial ref frame!!!
for i in range(len(vec['x'])):
    earth_x.append(float(vec['x'][i]))
    earth_y.append(float(vec['y'][i]))
    earth_z.append(float(vec['z'][i]))

In [56]:
# Check output looks sensible
vec

targetname,datetime_jd,datetime_str,M1,M2,k1,k2,phasecoeff,x,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,mag,mag,---,---,mag / deg,AU,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str25,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
67P/Churyumov-Gerasimenko,2459953.863937,A.D. 2023-Jan-09 08:44:04.1568,11.2,13.9,12.0,5.0,0.03,-3.444505659720326,-2.01529811172955,-0.7824777733431274,0.01361512061956163,-0.001640125884274181,-0.001023026664067215,0.02348749091349553,4.066732986656742,-0.01052233376238634
67P/Churyumov-Gerasimenko,2459956.868077,A.D. 2023-Jan-12 08:50:01.8528,11.2,13.9,12.0,5.0,0.03,-3.403978602868609,-2.019009305196668,-0.7850265415203742,0.01335816081538008,-0.0008328157860216082,-0.0006747539103341635,0.02330315049423218,4.034815432476373,-0.01072161335830465
67P/Churyumov-Gerasimenko,2459957.822871,A.D. 2023-Jan-13 07:44:56.0544,11.2,13.9,12.0,5.0,0.03,-3.391267457502642,-2.019683241381317,-0.7856184818099728,0.01326701767661173,-0.0005791162703836693,-0.0005652806996806055,0.02324386880187047,4.024551125628097,-0.01077841417826468
67P/Churyumov-Gerasimenko,2459964.842786,A.D. 2023-Jan-20 08:13:36.7104,11.2,13.9,12.0,5.0,0.03,-3.300839677733815,-2.017323136177724,-0.7868120079091985,0.01245369868918222,0.001235307693765872,0.0002185425969670557,0.02279992085942436,3.94768392220678,-0.01108792658197195
67P/Churyumov-Gerasimenko,2459966.871729,A.D. 2023-Jan-22 08:55:17.3856,11.2,13.9,12.0,5.0,0.03,-3.275855222497047,-2.014303930586379,-0.7861467618230418,0.0121708103463252,0.001738936136628179,0.000436405100539614,0.02266968158750547,3.925133691310624,-0.01113736255238601
67P/Churyumov-Gerasimenko,2459966.876288,A.D. 2023-Jan-22 09:01:51.2832,11.2,13.9,12.0,5.0,0.03,-3.275799737275682,-2.01429600022679,-0.7861447711491047,0.01217015096136869,0.001740054677198425,0.0004368890101460341,0.02266938833302884,3.925082915871988,-0.01113745147457623
67P/Churyumov-Gerasimenko,2459970.830448,A.D. 2023-Jan-26 07:55:50.7072,11.2,13.9,12.0,5.0,0.03,-3.228857768734552,-2.005529464626848,-0.7836014200889149,0.01156078626169196,0.002685687273778482,0.0008458551858015367,0.02241444519247415,3.880940879447828,-0.01117697335936618
67P/Churyumov-Gerasimenko,2459970.830859,A.D. 2023-Jan-26 07:56:26.2176,11.2,13.9,12.0,5.0,0.03,-3.228853017265195,-2.005528360789726,-0.7836010724339388,0.01156071923784386,0.002685782925363542,0.0008458965280223403,0.02241441866126721,3.880936285711749,-0.0111769736099083


In [57]:
# Now get the remaining 655-500 vectors
obj = Horizons(id='90000693', location='500', epochs=mjd[500:])

In [58]:
vec = obj.vectors(refplane='earth')  
for i in range(len(vec['x'])):
    earth_x.append(float(vec['x'][i]))
    earth_y.append(float(vec['y'][i]))
    earth_z.append(float(vec['z'][i]))

In [103]:
# Check this didn't miss anything or double anything
#earth_x[495:]


In [106]:
# Light time correct the MJDs:
delta = []
for i in range(len(earth_x)):
    delta.append(np.sqrt(earth_x[i]**2+earth_y[i]**2+earth_z[i]**2))

delta=np.array(delta)
mjd_ltc = np.zeros_like(mjd)

mjd_ltc = mjd-(((delta*km2au)/c)/(60**2*24))
mjd_ltc = np.ndarray.tolist(mjd_ltc)

In [107]:
namestr = path_67p+'test.txt'
mikkoWrite(namestr,mjd_ltc,np.ones_like(mjd),sun_x,sun_y,sun_z,earth_x,earth_y,earth_z,0.0)
# Go and type the number output rom this cell into the top of the file you've just created
# This is the number of lightcurves in total

326

In [109]:
mjd-mjd_ltc

array([0.02347158, 0.02328736, 0.02322812, 0.02278447, 0.02265432,
       0.02265403, 0.02239926, 0.02239923, 0.02239829, 0.02239823,
       0.02227098, 0.02220281, 0.02194854, 0.02188534, 0.02181949,
       0.02176036, 0.02151208, 0.02151185, 0.02151113, 0.02151108,
       0.02150835, 0.02114355, 0.0211426 , 0.02114167, 0.02108835,
       0.02108738, 0.02102646, 0.02102552, 0.02091202, 0.02091199,
       0.02091113, 0.02091111, 0.02074748, 0.02074746, 0.02074654,
       0.02074651, 0.0204836 , 0.02048278, 0.02043419, 0.0204334 ,
       0.02029592, 0.02029457, 0.02020277, 0.02016064, 0.01985701,
       0.01985645, 0.01985595, 0.01979506, 0.01979458, 0.01964115,
       0.01964065, 0.01964024, 0.01956967, 0.01956966, 0.01956943,
       0.01956942, 0.0195689 , 0.01956866, 0.01951227, 0.01951219,
       0.01950853, 0.01950848, 0.01950848, 0.01950844, 0.01950667,
       0.01950667, 0.01950663, 0.01954457, 0.01955648, 0.0195567 ,
       0.01955679, 0.01955701, 0.01957032, 0.01957057, 0.01964

In [24]:
sun_xyz = np.array([sun_x, sun_y, sun_z])
print(sun_xyz[:,0])
earth_xyz = np.array([earth_x, earth_y, earth_z])

[-3.75625676 -1.15950396 -0.41150245]


In [25]:
# Checking the phase angles: should make this into a function but too lazy
alpha_list_calc = []
for i in range(len(earth_xyz[1])):
    alpha_list_calc.append((360/(2.*np.pi))*np.arccos(np.dot(sun_xyz[:,i],earth_xyz[:,i])/(np.linalg.norm(sun_xyz[:,i])*np.linalg.norm(earth_xyz[:,i]))))

In [13]:
# This is just to grab ephemeris data to check phase angles
obj = Horizons(id='90000693', location='500', epochs=mjd[:500])
ephem_table = obj.ephemerides(quantities='43')

/Users/s1523386/anaconda3/lib/python3.8/site-packages/astroquery/jplhorizons/core.py:604: UserWarning: The uri used in this query is very long and might have been truncated. The results of the query might be compromised. If you queried a list of epochs, consider querying a range.
  warnings.warn(('The uri used in this query is very long '


In [31]:
#ephem_table

In [14]:
alpha_list = []
for row in ephem_table['alpha_true']:
    alpha_list.append(float(row))

In [15]:
obj = Horizons(id='90000693', location='geocentric', epochs=mjd[500:])
ephem_table = obj.ephemerides()

In [16]:
for row in ephem_table['alpha_true']:
    alpha_list.append(float(row))

In [37]:
#alpha_list

In [61]:
len(sun_x)

678

In [49]:
# Final step: read in intensities as calculated from shifts from Matlab script
# Obvs if you're using this script to create Mikko file in the first instance, you won't have these
intensities_file = path_67p+'67P_LSST_3AU_intensities.txt'
intensities_list = []
intf = open(intensities_file, 'r')
lines = intf.readlines()
for line in lines:
    data = line.split()
    intensities_list.append(float(data[0]))
intensities_list

[0.7426974474850968,
 0.8071534775260666,
 0.7681982761443079,
 0.9186711284470781,
 0.813416711500228,
 0.8266346497305266,
 0.790946714312474,
 0.7895765093782315,
 0.7488910041253953,
 0.7469548078516769,
 0.7820382891326926,
 0.8187811434530314,
 0.8681742405805063,
 0.9661252404818803,
 0.9673935587890998,
 0.7945107722304662,
 0.8541874888927949,
 0.8510288386780904,
 0.8337609263765468,
 0.8317849473090925,
 0.7243629306709622,
 0.7257023449347533,
 0.7496761743466329,
 0.7717249020134768,
 0.9839733495228667,
 0.9297859892538634,
 0.8931135955386051,
 0.8255391138221356,
 1.0374261011475687,
 1.0383394865185738,
 1.0448666849578,
 1.0461597604560826,
 0.8277611790179839,
 0.8273851741436872,
 0.8532837695269568,
 0.8534362832373643,
 0.9607382026140191,
 0.964231088496303,
 0.91218275326885,
 0.9400942082556291,
 1.1415977475110988,
 1.1298147340398295,
 1.162437778893275,
 0.999184697628772,
 0.981218609226745,
 1.013523342465471,
 1.0487374210794203,
 1.1436369244622586,
 1.0

In [79]:
# Code to write to lightcurve file in Mikko format
# Uncomment fouts/change to print if necessary

#fout = open(path_67p+'67P_LC_LSST_3AU_NU.dat', 'w+')
sameday = []
#fout.write('314\n')
counter = 1
for i, day in enumerate(mjd[:-1]):
    if (np.sqrt(sun_x[i]**2+sun_y[i]**2+sun_z[i]**2)>=3.5):
        newday = True
        sameday.append(day)
        if abs(day-mjd[i+1])<0.5:
            newday = False
        if newday==False:
            if day not in sameday:
                sameday.append(day)
        else:
            print('%i %i 1'%(len(sameday),counter))
            #fout.write('%i 1\n'%(len(sameday)))
            for time in sameday:
                loc = mjd.index(time)
                print('%.6f %.6e %.10f %.8f %.8f %.8f %.8f %.8f'%(time+2400000.5,1,sun_x[loc], sun_y[loc], sun_z[loc], earth_x[loc], earth_y[loc], earth_z[loc]))
            counter = counter+1
                #fout.write('%.6f %.6e %.10f %.8f %.8f %.8f %.8f %.8f\n'%(time+2400000.5,intensities_list[loc],sun_x[loc], sun_y[loc], sun_z[loc], earth_x[loc], earth_y[loc], earth_z[loc]))
            sameday = []
print('1 %i 1'%counter)
print('%.6f %.6e %.10f %.8f %.8f %.8f %.8f %.8f'%(mjd[-1]+2400000.5,1,sun_x[-1], sun_y[-1], sun_z[-1], earth_x[-1], earth_y[-1], earth_z[-1]))

#fout.write('1 1\n')
#fout.write('%.6f %.6e %.10f %.8f %.8f %.8f %.8f %.8f\n'%(mjd[-1]+2400000.5,intensities_list[-1],sun_x[-1], sun_y[-1], sun_z[-1], earth_x[-1], earth_y[-1], earth_z[-1]))

#fout.close()

1 1 1
2459953.863937 1.000000e+00 -3.7562567561 -1.15950396 -0.41150245 -3.44450566 -2.01529811 -0.78247777
1 2 1
2459956.868077 1.000000e+00 -3.7651157170 -1.17961553 -0.42115963 -3.40397860 -2.01900931 -0.78502654
1 3 1
2459957.822871 1.000000e+00 -3.7678975987 -1.18599697 -0.42422518 -3.39126746 -2.01968324 -0.78561848
1 4 1
2459964.842786 1.000000e+00 -3.7878567183 -1.23275814 -0.44670759 -3.30083968 -2.01732314 -0.78681201
2 5 1
2459966.871729 1.000000e+00 -3.7934649454 -1.24622154 -0.45318688 -3.27585522 -2.01430393 -0.78614676
2459966.876288 1.000000e+00 -3.7934774676 -1.24625177 -0.45320144 -3.27579974 -2.01429600 -0.78614477
4 6 1
2459970.830448 1.000000e+00 -3.8042025660 -1.27242288 -0.46580425 -3.22885777 -2.00552946 -0.78360142
2459970.830859 1.000000e+00 -3.8042036659 -1.27242560 -0.46580555 -3.22885302 -2.00552836 -0.78360107
2459970.845500 1.000000e+00 -3.8042428788 -1.27252233 -0.46585216 -3.22868377 -2.00548901 -0.78358868
2459970.846328 1.000000e+00 -3.8042450953 -1.2

In [ ]:
# Also making standard lightcurve output with dates, magnitudes and 0.01 mag. uncertainty

corr_fac = 16    # from mag_from_phase_func py script

#fout = open(path_67p+'67p_20210109_I11_R.dat', 'w+')
for i,day in enumerate(mjd):
    magnitude = corr_fac -2.5*np.log10(intensities_list[i])
    #fout.write('%.6f %.4f 0.01\n'%(day+2400000.5, magnitude))
#fout.close()